In [5]:
import requests
import zipfile
import io
import os
from dotenv import load_dotenv

load_dotenv()

def get_and_extract_document(api_key: str, rcept_no: str, extract_path: str):
    """
    공시서류원본파일 다운로드 & CP949 인코딩으로 압축 해제
    
    :param api_key: 발급받은 API 인증키 (40자리)
    :param rcept_no: 접수번호 (14자리)
    :param extract_path: 압축 풀 경로
    """
    url = "https://opendart.fss.or.kr/api/document.xml"
    params = {
        "crtfc_key": api_key,
        "rcept_no": rcept_no
    }

    # 디렉토리가 없으면 생성
    os.makedirs(extract_path, exist_ok=True)

    # ZIP 다운로드
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"HTTP 오류: {response.status_code}")

    # 메모리에서 ZIP 열기
    with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        for info in zf.infolist():
            # 파일명 CP949 복원
            filename = info.filename.encode('cp437').decode('cp949')
            # 파일 내용 읽기
            with zf.open(info) as source, open(f"{extract_path}/{filename}", "wb") as target:
                target.write(source.read())
    print(f"압축이 해제되었습니다: {extract_path}")

# 사용 예시
# get_and_extract_document("발급받은_API_KEY", "20250112000001", "./docs")


In [6]:
DART_API_KEY = os.getenv("DART_API_KEY")
get_and_extract_document(DART_API_KEY, "20250814002573", "./docs")

압축이 해제되었습니다: ./docs


In [7]:
import pandas as pd
import requests
from typing import Optional

def fetch_piic_decsn_df(
    api_key: str,
    corp_code: str,
    bgn_de: str,
    end_de: str,
    *,
    timeout: int = 15,
    save_path: Optional[str] = None
) -> pd.DataFrame:
    """
    DART '유상증자 결정' API 호출 후 응답 list를 DataFrame으로 반환.

    Parameters
    ----------
    api_key : str
        DART API 인증키(40자리)
    corp_code : str
        공시대상회사의 고유번호(8자리)
    bgn_de : str
        검색 시작일 (YYYYMMDD)
    end_de : str
        검색 종료일 (YYYYMMDD)
    timeout : int, optional
        요청 타임아웃(초), 기본 15
    save_path : str | None, optional
        CSV 저장 경로. 지정 시 UTF-8 BOM으로 저장.

    Returns
    -------
    pd.DataFrame
        공시 리스트 DataFrame
    """
    url = "https://opendart.fss.or.kr/api/pifricDecsn.json"
    params = {
        "crtfc_key": api_key,
        "corp_code": corp_code,
        "bgn_de": bgn_de,
        "end_de": end_de
    }

    resp = requests.get(url, params=params, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()

    if str(data.get("status")) != "000":
        raise ValueError(f"DART API 오류: {data.get('message')} (status={data.get('status')})")

    items = data.get("list", [])
    df = pd.DataFrame(items)

    # CSV 저장 옵션
    if save_path:
        df.to_csv(save_path, index=False, encoding="utf-8-sig")

    return df




In [9]:

df = fetch_piic_decsn_df(
    api_key=DART_API_KEY,
    corp_code="01109937",
    bgn_de="20250801",
    end_de="20250801",
    save_path="piicDecsn.csv"
)


In [10]:
df.head()

,rcept_no,corp_cls,corp_code,corp_name,piic_nstk_ostk_cnt,piic_nstk_estk_cnt,piic_fv_ps,piic_bfic_tisstk_ostk,piic_bfic_tisstk_estk,piic_fdpp_fclt,...,fric_nstk_dividrk,fric_nstk_dlprd,fric_nstk_lstprd,fric_bddd,fric_od_a_at_t,fric_od_a_at_b,fric_adt_a_atn,ssl_at,ssl_bgd,ssl_edd
0,20250801000893,K,01109937,티앤알바이오팹,"15,904,582",-,500,"21,546,828","474,683","1,700,000,000",...,2025년 01월 01일,-,2025년 11월 18일,2025년 08월 01일,2,-,참석,Y,20250802,20251015


In [8]:

import io
import zipfile
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from typing import Optional

def fetch_corp_codes_df(
    api_key: str,
    *,
    timeout: int = 30,
    save_csv_path: Optional[str] = None,
    save_zip_path: Optional[str] = None,   # ← ZIP 저장 옵션 추가
    save_xml_path: Optional[str] = None    # ← XML도 저장하고 싶을 때
) -> pd.DataFrame:
    """
    DART corpCode.xml(zip) 내려받아 XML 파싱 → DataFrame 반환.
    옵션으로 ZIP/내부 XML/CSV 저장 가능.
    """
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    params = {"crtfc_key": api_key}

    resp = requests.get(url, params=params, timeout=timeout)
    resp.raise_for_status()
    content = resp.content

    # 먼저 ZIP 시도
    try:
        zip_bytes = io.BytesIO(content)
        with zipfile.ZipFile(zip_bytes) as zf:
            xml_names = [n for n in zf.namelist() if n.lower().endswith(".xml")]
            if not xml_names:
                raise ValueError("ZIP 내부에 XML 파일이 없습니다.")
            # ZIP 저장 옵션
            if save_zip_path:
                with open(save_zip_path, "wb") as f:
                    f.write(zip_bytes.getvalue())

            with zf.open(xml_names[0]) as xf:
                xml_bytes = xf.read()
    except zipfile.BadZipFile:
        # ZIP이 아니면 보통 인증키 오류 등 에러 응답
        text = content.decode("utf-8", errors="ignore")
        raise ValueError(f"ZIP이 아닌 응답입니다. 인증키/쿼리를 확인하세요. 응답 예시: {text[:300]}")

    # XML 저장 옵션
    if save_xml_path:
        with open(save_xml_path, "wb") as f:
            f.write(xml_bytes)

    # XML 파싱 → DataFrame
    try:
        root = ET.fromstring(xml_bytes)
    except ET.ParseError:
        xml_text = xml_bytes.decode("utf-8", errors="ignore")
        root = ET.fromstring(xml_text)

    status = _find_text(root, ".//status")
    message = _find_text(root, ".//message")
    if status and status != "000":
        raise ValueError(f"DART API 오류(status={status}): {message}")

    corps = root.findall(".//list")
    rows = []
    for c in corps:
        rows.append({
            "corp_code": _find_text(c, "corp_code"),
            "corp_name": _find_text(c, "corp_name"),
            "corp_eng_name": _find_text(c, "corp_eng_name"),
            "stock_code": _find_text(c, "stock_code"),
            "modify_date": _find_text(c, "modify_date"),
        })
    df = pd.DataFrame(rows, columns=["corp_code","corp_name","corp_eng_name","stock_code","modify_date"])

    # CSV 저장 옵션
    if save_csv_path:
        df.to_csv(save_csv_path, index=False, encoding="utf-8-sig")

    return df


def _find_text(root: ET.Element, path: str) -> str:
    el = root.find(path)
    return el.text.strip() if (el is not None and el.text is not None) else ""

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

DART_API_KEY = os.getenv("DART_API_KEY")

df = fetch_corp_codes_df(
    api_key=DART_API_KEY,
    save_zip_path="corpCode.zip",      # ← ZIP 파일도 저장
    save_xml_path="corpCode.xml",      # ← 내부 XML 저장 (옵션)
    save_csv_path="corp_codes.csv"     # ← CSV 저장 (옵션)
)


In [3]:
import FinanceDataReader as fdr

df = fdr.StockListing("KRX")
df.to_csv("krx_listing.csv", index=False)

In [ ]:
KRX_LISTING = fdr.StockListing("KRX")[["Code", "ISU_CD", "Name", "Market"]]
# 삼성전자 종목 정보 조회

print("삼성전자 종목 정보:")
print(short_code)
print(stock_code)


삼성전자 종목 정보:
005930
KR7005930003
